In [91]:
#%matplotlib notebook

In [1]:
%matplotlib inline

In [2]:
import numpy as np
# -------------------------------------------------------------------------
# Const
# -------------------------------------------------------------------------
nm2m=1852. # 1 nautical mile to meters
kt2ms=nm2m/3600.  # knots to m/s
omega=2*np.pi/(3600.*24.) # angular speed omega=2pi*f(=frequency of earth : 1 cycle per day) 2pi* 1 / day in seconds
rhoa=1.15 #air density  Kg/m^3
radius=6378388 #137. # earth's radius according to WGS 84
deg2m=np.pi*radius/180.  # ds on cicle equals ds=r*dth - dth=pi/180
pn=101000.  # Atmospheric pressure [N/m^2] (101KPa - enviromental pressure)

tetaNE=45. #mean angle [degrees] of North Eastern quadrant
tetaNW=135. #        "              North Western
tetaSW=225. #        "              South West
tetaSE=315. #        "              South East

maxR=500.e3  # maximum radius of TC [m] (500Km)

In [3]:
from netCDF4 import Dataset

In [4]:
import glob

In [5]:
import re

In [6]:
import datetime

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import mplleaflet

In [8]:
def hvel(r):
    x=0.5+(r-rmaxh)/(maxR-rmaxh)*kh
    return (bh/rhoa*(rmaxh/r)**bh*dph*np.exp(-(rmaxh/r)**bh))**x

In [9]:
def pres(r):
    return pch+dph*np.exp(-(rmaxh/r)**bh)

In [10]:
def hvel2d(l1,l2,vtx,vty):
    r=np.sqrt(l1**2+l2**2)
    xh=0.5+(r-rmaxh)/(maxR-rmaxh)*kh
    ur=(bh/rhoa*(rmaxh/r)**bh*dph*np.exp(-(rmaxh/r)**bh))**xh
    theta=np.arctan2(l2,l1)
    ux=-ur*np.sin(theta)
    uy=ur*np.cos(theta)
    return ux+vtx,uy+vty, pres(r)

In [11]:
path='BestTrack/'

In [12]:
data=pd.read_csv(path+'Allstorms.ibtracs_all.v03r09.csv', header=1, low_memory=False)

In [13]:
data['Basin'] = data.loc[:,'Basin'].str.strip() # strip basin reference

In [14]:
data.head()

,Serial_Num,Season,Num,Basin,Sub_basin,Name,ISO_time,Nature,Latitude,Longitude,...,jtwc_??_wrad34_rad3,jtwc_??_wrad34_rad4,jtwc_??_wrad50_rad1,jtwc_??_wrad50_rad2,jtwc_??_wrad50_rad3,jtwc_??_wrad50_rad4,jtwc_??_wrad64_rad1,jtwc_??_wrad64_rad2,jtwc_??_wrad64_rad3,jtwc_??_wrad64_rad4
0,NaN,Year,#,BB,BB,NaN,YYYY-MM-DD HH:MM:SS,NaN,deg_north,deg_east,...,nmile,nmile,nmile,nmile,nmile,nmile,nmile,nmile,nmile,nmile
1,1842298N11080,1842,01,NI,BB,NOT NAMED,1842-10-25 06:00:00,NR,-999.,-999.,...,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000
2,1842298N11080,1842,01,NI,BB,NOT NAMED,1842-10-25 12:00:00,NR,-999.,-999.,...,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000
3,1842298N11080,1842,01,NI,AS,NOT NAMED,1842-10-25 18:00:00,NR,-999.,-999.,...,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000
4,1842298N11080,1842,01,NI,AS,NOT NAMED,1842-10-26 00:00:00,NR,-999.,-999.,...,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000


In [17]:
#storms=Dataset(path+'Allstorms.ibtracs_all.v03r09.nc')

In [18]:
#storms.variables.keys()

# ANALYSIS STARTS HERE

In [15]:
hurName='TOMAS'
year = '2010'

In [16]:
hurdata = data[data['Name'].str.contains(hurName,na=False) & data['ISO_time'].str.contains(year,na=False)]

In [17]:
hurdata.iloc[0]

Serial_Num                     2010069S12188
Season                                  2010
Num                                       10
Basin                                     SP
Sub_basin                                 MM
Name                                   TOMAS
ISO_time                 2010-03-10 00:00:00
Nature                                    NR
Latitude                              -12.30
Longitude                            -172.20
Wind(WMO)                               20.0
Pres(WMO)                             1005.0
Center                                  nadi
Wind(WMO) Percentile                   4.063
Pres(WMO) Percentile                   1.388
Track_type                              main
Latitude_for_mapping                  -12.30
Longitude_for_mapping                -172.20
Current Basin                             SP
hurdat_atl_lat                        -999.0
hurdat_atl_lon                        -999.0
hurdat_atl_grade                      -999.0
hurdat_atl

In [18]:
hurdata.shape

(96, 200)

In [19]:
ntc=hurdata['Basin'].unique()

In [20]:
ntc

array(['SP', 'NA'], dtype=object)

In [21]:
hur1=hurdata.loc[hurdata['Basin']=='NA'].copy()

In [22]:
hur1

,Serial_Num,Season,Num,Basin,Sub_basin,Name,ISO_time,Nature,Latitude,Longitude,...,jtwc_??_wrad34_rad3,jtwc_??_wrad34_rad4,jtwc_??_wrad50_rad1,jtwc_??_wrad50_rad2,jtwc_??_wrad50_rad3,jtwc_??_wrad50_rad4,jtwc_??_wrad64_rad1,jtwc_??_wrad64_rad2,jtwc_??_wrad64_rad3,jtwc_??_wrad64_rad4
327411,2010302N09306,2010,19,NA,NA,TOMAS,2010-10-29 06:00:00,TS,9.00,-53.70,...,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000
327412,2010302N09306,2010,19,NA,NA,TOMAS,2010-10-29 12:00:00,TS,9.80,-55.30,...,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000
327413,2010302N09306,2010,19,NA,NA,TOMAS,2010-10-29 18:00:00,TS,10.80,-56.80,...,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000
327414,2010302N09306,2010,19,NA,NA,TOMAS,2010-10-30 00:00:00,TS,11.90,-57.80,...,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000
327415,2010302N09306,2010,19,NA,NA,TOMAS,2010-10-30 06:00:00,TS,12.70,-58.90,...,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000
327416,2010302N09306,2010,19,NA,CS,TOMAS,2010-10-30 09:00:00,TS,13.00,-59.50,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
327417,2010302N09306,2010,19,NA,CS,TOMAS,2010-10-30 12:00:00,TS,13.10,-60.10,...,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000
327418,2010302N09306,2010,19,NA,CS,TOMAS,2010-10-30 18:00:00,TS,13.30,-61.00,...,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000
327419,2010302N09306,2010,19,NA,CS,TOMAS,2010-10-30 20:00:00,TS,13.40,-61.20,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
327420,2010302N09306,2010,19,NA,CS,TOMAS,2010-10-31 00:00:00,TS,13.50,-61.70,...,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000,-999.000


In [23]:
hur1.dropna()

,Serial_Num,Season,Num,Basin,Sub_basin,Name,ISO_time,Nature,Latitude,Longitude,...,jtwc_??_wrad34_rad3,jtwc_??_wrad34_rad4,jtwc_??_wrad50_rad1,jtwc_??_wrad50_rad2,jtwc_??_wrad50_rad3,jtwc_??_wrad50_rad4,jtwc_??_wrad64_rad1,jtwc_??_wrad64_rad2,jtwc_??_wrad64_rad3,jtwc_??_wrad64_rad4
327416,2010302N09306,2010,19,NA,CS,TOMAS,2010-10-30 09:00:00,TS,13.00,-59.50,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
327419,2010302N09306,2010,19,NA,CS,TOMAS,2010-10-30 20:00:00,TS,13.40,-61.20,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
327446,2010302N09306,2010,19,NA,NA,TOMAS,2010-11-06 07:30:00,TS,21.70,-71.60,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0


In [24]:
hur1[['Longitude','Latitude']]=hur1.loc[:,['Longitude','Latitude']].apply(pd.to_numeric)

Check if we cross International Date Line (IDL)

In [25]:
sig=np.sign(hur1.Longitude.values)
sig1=sig[0]
m=sig != sig1

In [26]:
if sum(m)>0:
# adjust the lon values going from -180:180
        if sig1 > 0:
                hur1.loc[hur1.Longitude < 0, ['Longitude']] = hur1.Longitude+360.
        elif sig1 < 0:
                hur1.loc[hur1.Longitude > 0, ['Longitude']] = hur1.Longitude-360.



In [27]:
plt.plot(hur1['Longitude'],hur1['Latitude'],'o--')
mplleaflet.display()

In [28]:
idf=hur1.iloc[0].Serial_Num

In [29]:
filename=glob.glob('BestTrack/ibtracs/{}*'.format(idf))

In [30]:
dat=Dataset(filename[0])

In [31]:
dat.variables.keys()

[u'nobs_wmo',
 u'nobs',
 u'genesis_basin',
 u'num_basins',
 u'season',
 u'source',
 u'track_from_source',
 u'name_from_source',
 u'time_wmo',
 u'isotime_wmo',
 u'source_wmo',
 u'averaging_time',
 u'lat_wmo',
 u'lon_wmo',
 u'wind_wmo',
 u'pres_wmo',
 u'basin_wmo',
 u'sub_basin_wmo',
 u'nature_wmo',
 u'dist2land_wmo',
 u'landfall_wmo',
 u'pres_perc_wmo',
 u'wind_perc_wmo',
 u'time',
 u'isotime',
 u'lat_for_mapping',
 u'lon_for_mapping',
 u'wind_for_mapping',
 u'pres_for_mapping',
 u'nature_for_mapping',
 u'dist2land',
 u'landfall',
 u'lat_from_source',
 u'lon_from_source',
 u'wind_from_source',
 u'wind_avg_period',
 u'wind_perc_from_source',
 u'pres_from_source',
 u'pres_perc_from_source',
 u'basin',
 u'sub_basin',
 u'pos_stdev',
 u'pos_num',
 u'pos_flag',
 u'wind_quality',
 u'wind_stdev',
 u'wind_condition',
 u'wind_num',
 u'wind_flag',
 u'wind_maximum',
 u'wind_minimum',
 u'wind_median',
 u'pres_quality',
 u'pres_stdev',
 u'pres_condition',
 u'pres_flag',
 u'pres_num',
 u'pres_maximum'

In [32]:
wradii=dat['atcf_wrad']

In [33]:
wradiinp=np.array(wradii)

In [34]:
w34= wradiinp[:,0,:]

In [35]:
w34ne=w34[:,0]
w34nw=w34[:,1]
w34se=w34[:,2]
w34sw=w34[:,3]

In [36]:
w50=wradiinp[:,1,:]

In [37]:
w50ne=w50[:,0]
w50nw=w50[:,1]
w50se=w50[:,2]
w50sw=w50[:,3]

In [38]:
w64=wradiinp[:,2,:]

In [39]:
w64ne=w64[:,0]
w64nw=w64[:,1]
w64se=w64[:,2]
w64sw=w64[:,3]

In [40]:
isot=dat['isotime']

In [41]:
isot1=[''.join(x) for x in isot]

In [42]:
time=[re.sub(r'[-: ]+','',x)[:-4] for x in isot1]

In [43]:
lat=dat['atcf_lat'][:]
lon=dat['atcf_lon'][:]
pcenter=dat['atcf_pres'][:]
vmaxh=dat['atcf_wind'][:] # 10 minute wind in Knots
rmaxh=dat['atcf_rmw'][:]*nm2m

In [44]:
dic={'t':time, 'lat':lat,'lon':lon,'pcenter':pcenter,'vmax':vmaxh, 'rmax':rmaxh, '34ne': w34ne, '34nw': w34nw, '34se':w34se, '34sw':w34sw, '50ne': w50ne, '50nw': w50nw, '50se':w50se, '50sw':w50sw,'64ne': w64ne, '64nw': w64nw, '64se':w64se, '64sw':w64sw} 

In [45]:
inpData=pd.DataFrame(dic)

In [46]:
inpData=inpData.apply(pd.to_numeric)

In [47]:
inpData

,34ne,34nw,34se,34sw,50ne,50nw,50se,50sw,64ne,64nw,64se,64sw,lat,lon,pcenter,rmax,t,vmax
0,0,0,0,0,0,0,0,0,0,0,0,0,9.000000,-53.700001,1006,222240.0,2010102906,30
1,75,40,0,75,0,0,0,0,0,0,0,0,9.800000,-55.299999,1003,166680.0,2010102912,35
2,90,50,50,90,0,0,0,0,0,0,0,0,10.800000,-56.799999,998,55560.0,2010102918,45
3,90,50,50,90,30,0,0,30,0,0,0,0,11.900000,-57.799999,999,55560.0,2010103000,55
4,90,50,50,90,30,0,0,30,0,0,0,0,12.700000,-58.900002,997,55560.0,2010103006,60
5,120,60,50,90,50,30,20,40,20,20,0,0,13.100000,-60.099998,993,37040.0,2010103012,65
6,120,60,50,90,50,30,20,40,20,20,0,0,13.300000,-61.000000,990,37040.0,2010103018,80
7,120,60,50,90,50,30,20,40,30,20,10,20,13.500000,-61.700001,982,37040.0,2010103100,85
8,120,60,50,90,60,30,20,50,30,20,10,20,13.800000,-62.400002,983,27780.0,2010103106,85
9,120,60,50,90,60,30,20,50,30,20,10,20,14.000000,-63.299999,988,27780.0,2010103112,80


In [48]:
inpData.loc[inpData['lat'] == 9.969210e+36, 'lat'] = np.nan
inpData.loc[inpData['lon'] == 9.969210e+36, 'lon'] = np.nan
inpData.loc[inpData['pcenter'] == -32767., 'pcenter'] = np.nan
inpData.loc[inpData['rmax'] == -60684484., 'rmax'] = np.nan
inpData.loc[inpData['vmax'] == -32767, 'vmax'] = np.nan
inpData.loc[inpData['34ne'] == -32767, '34ne']=np.nan
inpData.loc[inpData['34nw'] == -32767, '34nw']=np.nan
inpData.loc[inpData['34se'] == -32767, '34se']=np.nan
inpData.loc[inpData['34sw'] == -32767, '34sw']=np.nan
inpData.loc[inpData['50ne'] == -32767, '50ne']=np.nan
inpData.loc[inpData['50nw'] == -32767, '50nw']=np.nan
inpData.loc[inpData['50se'] == -32767, '50se']=np.nan
inpData.loc[inpData['50sw'] == -32767, '50sw']=np.nan
inpData.loc[inpData['64ne'] == -32767, '64ne']=np.nan
inpData.loc[inpData['64nw'] == -32767, '64nw']=np.nan
inpData.loc[inpData['64se'] == -32767, '64se']=np.nan
inpData.loc[inpData['64sw'] == -32767, '64sw']=np.nan

In [49]:
inpData

,34ne,34nw,34se,34sw,50ne,50nw,50se,50sw,64ne,64nw,64se,64sw,lat,lon,pcenter,rmax,t,vmax
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.000000,-53.700001,1006.0,222240.0,2010102906,30.0
1,75.0,40.0,0.0,75.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.800000,-55.299999,1003.0,166680.0,2010102912,35.0
2,90.0,50.0,50.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.800000,-56.799999,998.0,55560.0,2010102918,45.0
3,90.0,50.0,50.0,90.0,30.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0,11.900000,-57.799999,999.0,55560.0,2010103000,55.0
4,90.0,50.0,50.0,90.0,30.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0,12.700000,-58.900002,997.0,55560.0,2010103006,60.0
5,120.0,60.0,50.0,90.0,50.0,30.0,20.0,40.0,20.0,20.0,0.0,0.0,13.100000,-60.099998,993.0,37040.0,2010103012,65.0
6,120.0,60.0,50.0,90.0,50.0,30.0,20.0,40.0,20.0,20.0,0.0,0.0,13.300000,-61.000000,990.0,37040.0,2010103018,80.0
7,120.0,60.0,50.0,90.0,50.0,30.0,20.0,40.0,30.0,20.0,10.0,20.0,13.500000,-61.700001,982.0,37040.0,2010103100,85.0
8,120.0,60.0,50.0,90.0,60.0,30.0,20.0,50.0,30.0,20.0,10.0,20.0,13.800000,-62.400002,983.0,27780.0,2010103106,85.0
9,120.0,60.0,50.0,90.0,60.0,30.0,20.0,50.0,30.0,20.0,10.0,20.0,14.000000,-63.299999,988.0,27780.0,2010103112,80.0


In [50]:
inpData.loc[inpData['34ne'] == -1, '34ne']=0
inpData.loc[inpData['34nw'] == -1, '34nw']=0
inpData.loc[inpData['34se'] == -1, '34se']=0
inpData.loc[inpData['34sw'] == -1, '34sw']=0
inpData.loc[inpData['50ne'] == -1, '50ne']=0
inpData.loc[inpData['50nw'] == -1, '50nw']=0
inpData.loc[inpData['50se'] == -1, '50se']=0
inpData.loc[inpData['50sw'] == -1, '50sw']=0
inpData.loc[inpData['64ne'] == -1, '64ne']=0
inpData.loc[inpData['64nw'] == -1, '64nw']=0
inpData.loc[inpData['64se'] == -1, '64se']=0
inpData.loc[inpData['64sw'] == -1, '64sw']=0

In [51]:
inpData=inpData.dropna()

In [52]:
inpData

,34ne,34nw,34se,34sw,50ne,50nw,50se,50sw,64ne,64nw,64se,64sw,lat,lon,pcenter,rmax,t,vmax
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.000000,-53.700001,1006.0,222240.0,2010102906,30.0
1,75.0,40.0,0.0,75.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.800000,-55.299999,1003.0,166680.0,2010102912,35.0
2,90.0,50.0,50.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.800000,-56.799999,998.0,55560.0,2010102918,45.0
3,90.0,50.0,50.0,90.0,30.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0,11.900000,-57.799999,999.0,55560.0,2010103000,55.0
4,90.0,50.0,50.0,90.0,30.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0,12.700000,-58.900002,997.0,55560.0,2010103006,60.0
5,120.0,60.0,50.0,90.0,50.0,30.0,20.0,40.0,20.0,20.0,0.0,0.0,13.100000,-60.099998,993.0,37040.0,2010103012,65.0
6,120.0,60.0,50.0,90.0,50.0,30.0,20.0,40.0,20.0,20.0,0.0,0.0,13.300000,-61.000000,990.0,37040.0,2010103018,80.0
7,120.0,60.0,50.0,90.0,50.0,30.0,20.0,40.0,30.0,20.0,10.0,20.0,13.500000,-61.700001,982.0,37040.0,2010103100,85.0
8,120.0,60.0,50.0,90.0,60.0,30.0,20.0,50.0,30.0,20.0,10.0,20.0,13.800000,-62.400002,983.0,27780.0,2010103106,85.0
9,120.0,60.0,50.0,90.0,60.0,30.0,20.0,50.0,30.0,20.0,10.0,20.0,14.000000,-63.299999,988.0,27780.0,2010103112,80.0


In [53]:
dph=pn-inpData.pcenter*100.

In [89]:
dph

0      400.0
1      700.0
2     1200.0
3     1100.0
4     1300.0
5     1700.0
6     2000.0
7     2800.0
8     2700.0
9     2200.0
10    1600.0
11    1300.0
12     700.0
13     500.0
14     500.0
15     500.0
16     700.0
17     600.0
18     400.0
19     400.0
20     400.0
21     400.0
22     600.0
23     700.0
24    1200.0
25    1400.0
26    1500.0
27    2100.0
28    2500.0
29    2300.0
30    1800.0
31    1500.0
32    1800.0
33    2000.0
34    2200.0
35    2300.0
36    2000.0
37    1800.0
38    1800.0
39    1800.0
40    1800.0
41    1800.0
42    1700.0
43    1700.0
44    1600.0
45    1600.0
46    1600.0
47    1600.0
48    1600.0
49    1500.0
50    1500.0
Name: pcenter, dtype: float64

In [90]:
inpData=inpData.assign(dp=dph)

In [91]:
inpData.iloc[3]

34ne       9.000000e+01
34nw       5.000000e+01
34se       5.000000e+01
34sw       9.000000e+01
50ne       3.000000e+01
50nw       0.000000e+00
50se       0.000000e+00
50sw       3.000000e+01
64ne       0.000000e+00
64nw       0.000000e+00
64se       0.000000e+00
64sw       0.000000e+00
lat        1.190000e+01
lon       -5.780000e+01
pcenter    9.990000e+02
rmax       5.556000e+04
t          2.010103e+09
vmax       5.500000e+01
dp         1.100000e+03
Name: 3, dtype: float64

### create inpData file

In [54]:
inpData['t']=pd.to_datetime(inpData['t'],format='%Y%m%d%H')

In [55]:
inpData.head()

,34ne,34nw,34se,34sw,50ne,50nw,50se,50sw,64ne,64nw,64se,64sw,lat,lon,pcenter,rmax,t,vmax
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,-53.700001,1006.0,222240.0,2010-10-29 06:00:00,30.0
1,75.0,40.0,0.0,75.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.8,-55.299999,1003.0,166680.0,2010-10-29 12:00:00,35.0
2,90.0,50.0,50.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.8,-56.799999,998.0,55560.0,2010-10-29 18:00:00,45.0
3,90.0,50.0,50.0,90.0,30.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0,11.9,-57.799999,999.0,55560.0,2010-10-30 00:00:00,55.0
4,90.0,50.0,50.0,90.0,30.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0,12.7,-58.900002,997.0,55560.0,2010-10-30 06:00:00,60.0


In [94]:
inpData.t.iloc[0]

Timestamp('2010-10-29 06:00:00')

In [95]:
inpData['time']=inpData['t']-inpData.t.iloc[0]

In [96]:
inpData['time']=inpData['time'] / pd.Timedelta('1 hour')

In [97]:
inpData=inpData.set_index('time')
inpData

,34ne,34nw,34se,34sw,50ne,50nw,50se,50sw,64ne,64nw,64se,64sw,lat,lon,pcenter,rmax,t,vmax,dp
time,,,,,,,,,,,,,,,,,,,
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.000000,-53.700001,1006.0,222240.0,2010-10-29 06:00:00,30.0,400.0
6.0,75.0,40.0,0.0,75.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.800000,-55.299999,1003.0,166680.0,2010-10-29 12:00:00,35.0,700.0
12.0,90.0,50.0,50.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.800000,-56.799999,998.0,55560.0,2010-10-29 18:00:00,45.0,1200.0
18.0,90.0,50.0,50.0,90.0,30.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0,11.900000,-57.799999,999.0,55560.0,2010-10-30 00:00:00,55.0,1100.0
24.0,90.0,50.0,50.0,90.0,30.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0,12.700000,-58.900002,997.0,55560.0,2010-10-30 06:00:00,60.0,1300.0
30.0,120.0,60.0,50.0,90.0,50.0,30.0,20.0,40.0,20.0,20.0,0.0,0.0,13.100000,-60.099998,993.0,37040.0,2010-10-30 12:00:00,65.0,1700.0
36.0,120.0,60.0,50.0,90.0,50.0,30.0,20.0,40.0,20.0,20.0,0.0,0.0,13.300000,-61.000000,990.0,37040.0,2010-10-30 18:00:00,80.0,2000.0
42.0,120.0,60.0,50.0,90.0,50.0,30.0,20.0,40.0,30.0,20.0,10.0,20.0,13.500000,-61.700001,982.0,37040.0,2010-10-31 00:00:00,85.0,2800.0
48.0,120.0,60.0,50.0,90.0,60.0,30.0,20.0,50.0,30.0,20.0,10.0,20.0,13.800000,-62.400002,983.0,27780.0,2010-10-31 06:00:00,85.0,2700.0


## convert to SI

In [98]:
inpData['vmax']=inpData['vmax']*kt2ms

In [99]:
inpData[['64ne','64se','64sw','64nw','50ne','50se','50sw','50nw','34ne','34se','34sw','34nw']]=inpData[['64ne','64se','64sw','64nw','50ne','50se','50sw','50nw','34ne','34se','34sw','34nw']]*nm2m

In [100]:
inpData

,34ne,34nw,34se,34sw,50ne,50nw,50se,50sw,64ne,64nw,64se,64sw,lat,lon,pcenter,rmax,t,vmax,dp
time,,,,,,,,,,,,,,,,,,,
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.000000,-53.700001,1006.0,222240.0,2010-10-29 06:00:00,15.433333,400.0
6.0,138900.0,74080.0,0.0,138900.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.800000,-55.299999,1003.0,166680.0,2010-10-29 12:00:00,18.005556,700.0
12.0,166680.0,92600.0,92600.0,166680.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.800000,-56.799999,998.0,55560.0,2010-10-29 18:00:00,23.150000,1200.0
18.0,166680.0,92600.0,92600.0,166680.0,55560.0,0.0,0.0,55560.0,0.0,0.0,0.0,0.0,11.900000,-57.799999,999.0,55560.0,2010-10-30 00:00:00,28.294444,1100.0
24.0,166680.0,92600.0,92600.0,166680.0,55560.0,0.0,0.0,55560.0,0.0,0.0,0.0,0.0,12.700000,-58.900002,997.0,55560.0,2010-10-30 06:00:00,30.866667,1300.0
30.0,222240.0,111120.0,92600.0,166680.0,92600.0,55560.0,37040.0,74080.0,37040.0,37040.0,0.0,0.0,13.100000,-60.099998,993.0,37040.0,2010-10-30 12:00:00,33.438889,1700.0
36.0,222240.0,111120.0,92600.0,166680.0,92600.0,55560.0,37040.0,74080.0,37040.0,37040.0,0.0,0.0,13.300000,-61.000000,990.0,37040.0,2010-10-30 18:00:00,41.155556,2000.0
42.0,222240.0,111120.0,92600.0,166680.0,92600.0,55560.0,37040.0,74080.0,55560.0,37040.0,18520.0,37040.0,13.500000,-61.700001,982.0,37040.0,2010-10-31 00:00:00,43.727778,2800.0
48.0,222240.0,111120.0,92600.0,166680.0,111120.0,55560.0,37040.0,92600.0,55560.0,37040.0,18520.0,37040.0,13.800000,-62.400002,983.0,27780.0,2010-10-31 06:00:00,43.727778,2700.0


## save inpData file

In [101]:
column_order=['lat','lon','dp','vmax','64ne','64se','64sw','64nw','50ne','50se','50sw','50nw','34ne','34se','34sw','34nw']

In [102]:
header=['lat','long','dp','vmax','64ne','64se','64sw','64nw','50ne','50se','50sw','50nw','34ne','34se','34sw','34nw']

In [153]:
inpData.to_csv('tmp/inpDataBT.txt',index=True, columns=column_order, sep='\t', header=header)

### create bulInfo.txt file

In [62]:
tt=pd.to_datetime(inpData['t'][0])

In [63]:
tt=datetime.datetime.strftime(tt,'%d %b %Y %H:%M:%S')

In [64]:
dic0={'advNo':[1],'tShift':[0],'$date':tt,'land':[1],'notes':[0]}

In [65]:
bul=pd.DataFrame.from_dict(dic0)

In [66]:
bul

,$date,advNo,land,notes,tShift
0,29 Oct 2010 06:00:00,1,1,0,0


In [76]:
bul.to_csv('tmp/bulInfo.txt',index=False, columns=['advNo','tShift','$date','land','notes'], sep='\t')

### create info.xml file

In [79]:
import xml.etree.ElementTree as et 
from xml.dom import minidom

def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = et.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")

In [107]:
info = et.Element('setexp')
et.SubElement(info, 'source').text = 'Tropical Cyclone Bulletin through GDACS/PDC'
et.SubElement(info, 'hurName').text = hurName
et.SubElement(info, 'hurId').text = hurName
et.SubElement(info, 'basin').text = ntc[0].strip()
et.SubElement(info, 'bulNo').text = '1'
et.SubElement(info, 'bulDate').text = tt
et.SubElement(info, 'n').text = '100000'
et.SubElement(info, 'fk').text = '0.81'
et.SubElement(info, 'stormsurge').text = '0'
et.SubElement(info, 'timefactor').text = '1'
et.SubElement(info, 'landfall').text = '1'

In [116]:
xmlf = minidom.parseString(prettify(info))

In [117]:
with open('tmp/info.xml','w') as f:
      xmlf.writexml(f)

The next step is to substract translational and coriolis velocity, see [Subtract translational and Coriolis velocity.ipynb](./Subtract translational and Coriolis velocity.ipynb).